In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

class DataCleaner:
    def __init__(self, df: pd.DataFrame):
        """Inicializa la clase con un DataFrame."""
        self.df = df.copy()
        self.numeric_vars = []
        self.categorical_vars = []

    def drop_duplicates(self):
        """Elimina filas duplicadas."""
        self.df.drop_duplicates(inplace=True)
        return self

    def fill_missing(self, column, method="mean", value=None):
        """Rellena valores nulos en una columna específica."""
        if pd.api.types.is_numeric_dtype(self.df[column]):
            if method == "mean":
                self.df[column].fillna(self.df[column].mean(), inplace=True)
            elif method == "median":
                self.df[column].fillna(self.df[column].median(), inplace=True)
            elif method == "value" and value is not None:
                self.df[column].fillna(value, inplace=True)
        else:  
            self.df[column].fillna(value if value is not None else self.df[column].mode()[0], inplace=True)
        return self

    def drop_columns(self, columns):
        """Elimina las columnas especificadas."""
        self.df.drop(columns=columns, inplace=True, errors="ignore")
        return self

    def normalize_text(self, column, remove_specials=False):
        """Normaliza texto en minúsculas y elimina caracteres especiales (opcional)."""
        self.df[column] = self.df[column].astype(str).str.lower().str.strip()
        
        return self
    
    def separate_variables(self):
        """Separa columnas en numéricas y categóricas."""
        self.numeric_vars = self.df.select_dtypes(include=['number']).columns.tolist()
        self.categorical_vars = self.df.select_dtypes(exclude=['number']).columns.tolist()
        return self

    def describe_numeric(self):
        """Devuelve un resumen de las variables numéricas."""
        return self.df[self.numeric_vars].describe() if self.numeric_vars else "No hay variables numéricas."

    def describe_categorical(self):
        """Resumen de variables categóricas con sus frecuencias."""
        if not self.categorical_vars:
            return "No hay variables categóricas."
        return {col: self.df[col].value_counts() for col in self.categorical_vars}

    def get_info(self):
        """Devuelve información del DataFrame como un string."""
        buffer = io.StringIO()
        self.df.info(buf=buffer)
        return buffer.getvalue()

    def get_cleaned_data(self):
        """Devuelve el DataFrame limpio."""
        return self.df
    
    ### 📊 FUNCIONES DE VISUALIZACIÓN ###
    
    def plot_missing_values(self):
        """Muestra un gráfico de barras con el porcentaje de valores nulos por columna."""
        missing = self.df.isnull().mean() * 100
        missing = missing[missing > 0]
        if missing.empty:
            print("No hay valores nulos en el DataFrame.")
            return
        plt.figure(figsize=(10, 5))
        missing.sort_values().plot(kind="barh", color="salmon")
        plt.xlabel("Porcentaje de valores nulos")
        plt.ylabel("Columnas")
        plt.title("Valores nulos por columna")
        plt.show()

    def plot_numeric_distributions(self):
        """Muestra histogramas de las variables numéricas."""
        if not self.numeric_vars:
            print("No hay variables numéricas para graficar.")
            return
        self.df[self.numeric_vars].hist(figsize=(12, 8), bins=20, color="skyblue", edgecolor="black")
        plt.suptitle("Distribución de variables numéricas")
        plt.show()

    def plot_categorical_counts(self, top_n=10):
        """Muestra gráficos de barras para las variables categóricas más frecuentes."""
        if not self.categorical_vars:
            print("No hay variables categóricas para graficar.")
            return
        for col in self.categorical_vars:
            plt.figure(figsize=(10, 4))
            self.df[col].value_counts().nlargest(top_n).plot(kind="bar", color="lightcoral", edgecolor="black")
            plt.title(f"Frecuencia de valores en {col}")
            plt.xlabel(col)
            plt.ylabel("Frecuencia")
            plt.xticks(rotation=45)
            plt.show()

    def plot_correlation_matrix(self):
        """Muestra una matriz de correlación para variables numéricas."""
        if len(self.numeric_vars) < 2:
            print("No hay suficientes variables numéricas para calcular la correlación.")
            return
        plt.figure(figsize=(10, 6))
        sns.heatmap(self.df[self.numeric_vars].corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
        plt.title("Matriz de correlación")
        plt.show()

    def plot_boxplots(self):
        """Muestra boxplots para detectar valores atípicos en variables numéricas."""
        if not self.numeric_vars:
            print("No hay variables numéricas para graficar.")
            return
        plt.figure(figsize=(12, 6))
        self.df[self.numeric_vars].boxplot(rot=45)
        plt.title("Boxplots de variables numéricas")
        plt.ylabel("Valores")
        plt.show()
